<a href="https://colab.research.google.com/github/YoshiyukiKono/gen_ai-sandbox/blob/main/langchain_few_shot_astradb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install faiss-cpu
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00


In [25]:
!pip install cassandra-driver
!pip install cassio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 87.4 MB/s eta 0:00:00


In [7]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.9 MB/s eta 0:00:00


In [26]:
from langchain.vectorstores import Cassandra

In [27]:
import os
import getpass

database_mode = (input("\n(C)assandra or (A)stra DB? ")).upper()

keyspace_name = input("\nKeyspace name? ")

if database_mode == "A":
    ASTRA_DB_APPLICATION_TOKEN = getpass.getpass('\nAstra DB Token ("AstraCS:...") ')
    #
    ASTRA_DB_SECURE_BUNDLE_PATH = input("Full path to your Secure Connect Bundle? ")
elif database_mode == "C":
    CASSANDRA_CONTACT_POINTS = input(
        "Contact points? (comma-separated, empty for localhost) "
    ).strip()


(C)assandra or (A)stra DB? A

Keyspace name? demo

Astra DB Token ("AstraCS:...") ··········


KeyboardInterrupt: ignored

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

if database_mode == "C":
    if CASSANDRA_CONTACT_POINTS:
        cluster = Cluster(
            [cp.strip() for cp in CASSANDRA_CONTACT_POINTS.split(",") if cp.strip()]
        )
    else:
        cluster = Cluster()
    session = cluster.connect()
elif database_mode == "A":
    ASTRA_DB_CLIENT_ID = "token"
    cluster = Cluster(
        cloud={
            "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
        },
        auth_provider=PlainTextAuthProvider(
            ASTRA_DB_CLIENT_ID,
            ASTRA_DB_APPLICATION_TOKEN,
        ),
    )
    session = cluster.connect()
else:
    raise NotImplementedError

In [28]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../../../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = OpenAIEmbeddings()

RuntimeError: ignored

In [ ]:
table_name = "my_vector_db_table"

docsearch = Cassandra.from_documents(
    documents=docs,
    embedding=embedding_function,
    session=session,
    keyspace=keyspace_name,
    table_name=table_name,
)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [ ]:
table_name = "my_vector_db_table"
#vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)
vectorstore = Cassandra.from_texts(
    to_vectorize,
    embeddings,
    metadatas=examples
    session=session,
    keyspace=keyspace_name,
    table_name=table_name,
)

https://python.langchain.com/docs/integrations/vectorstores/cassandra

In [19]:
import getpass
my_key = getpass.getpass()

··········


In [21]:
import os
import openai

# APIキーを取得
openai.api_key = my_key

# 質問の返答をresに代入
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "YMOについて教えてください"},
    ],
)
# 返答を出力
print(res.choices[0]["message"]["content"])

YMO（Yellow Magic Orchestra）は、1978年に日本で結成された電子音楽バンドです。彼らは日本の音楽史上、初めてコンピュータを使用して音楽を制作したことで知られています。

YMOのメンバーは、坂本龍一（キーボード、シンセサイザー、ピアノ）、高橋幸宏（ボーカル、ドラムス、シンセサイザー）、細野晴臣（ベース、シンセサイザー）の3人です。彼らは個々にも成功を収めており、YMO以前から音楽活動を行っていました。

YMOのサウンドは、テクノポップ、エレクトロポップ、シンセポップなどのジャンルに分類されます。彼らは当時の最先端の音楽技術を駆使し、独自のサウンドを創り出しました。特に、コンピュータが持つリズムや効果音を活用したことで、その音楽は革新的であり、世界中で高く評価されました。

YMOは、日本国内外で大きな成功を収めました。彼らの代表曲には、「コンピューター・ゲームのテーマ」、「テクノポリス」、「コズミック・サーフィン」などがあります。YMOの活動は1983年に一度一旦終了しましたが、その後もメンバーはソロ活動や他のプロジェクトで音楽活動を続けています。

YMOは、日本の音楽シーンにおいて電子音楽の先駆者であり、後の世代のアーティストやバンドに多大な影響を与えました。彼らの功績は広く認められ、2003年にはロックの殿堂入りを果たしました。

YMOの音楽は、クールでユニークなサウンドを持っており、今もなお多くの人に愛され続けています。


In [22]:
import os
import openai

# APIキーを取得
openai.api_key = my_key

# 質問の返答をresに代入
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "3足す３掛ける３は何ですか？"}#3足す３に3を掛けるといくつですか？ OR 3足す３掛ける３は何ですか？
    ],
)
# 返答を出力
print(res.choices[0]["message"]["content"])

3足す3は6です。3を3倍すると9になります。


In [23]:
import os
import openai

# APIキーを取得
openai.api_key = my_key

# 質問の返答をresに代入
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "3足す３に3を掛けるといくつですか？"}#3足す３に3を掛けるといくつですか？ OR 3足す３掛ける３は何ですか？
    ],
)
# 返答を出力
print(res.choices[0]["message"]["content"])

3足す3は6です。6に3を掛けると18になります。


In [24]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)
import openai
openai.api_key = my_key
# few shot promptingの定義
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "1足す1に2を掛けるといくつですか？", "output": "4"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {"input": "Write me a poem about the moon",
     "output": "One for the moon, and one for me, who are we to talk about the moon?",},
]

# データの入力と出力を連結してベクトル化のためのリストを作成
to_vectorize = [" ".join(example.values()) for example in examples]

# OpenAIの埋め込みモデルを初期化
embeddings = OpenAIEmbeddings(openai_api_key=my_key)

# 教師データをベクトル化してChromaベクトルストアに保存
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

# セマンティック類似性に基づく選択器を初期化。kは類似する最上位kの例を選択する数を意味する
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# メッセージプロンプトテンプレートを定義
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

# 最終的なチャットプロンプトテンプレートを定義
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# チャットモデルとプロンプトテンプレートをチェーン化
chain = final_prompt | ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo",openai_api_key=my_key)

# チェーンを使って質問を投げ、結果を表示
#print(chain.invoke({"input": "What's 3+3?"}).content)
print(chain.invoke({"input": "3足す３掛ける３は何ですか？"}).content)

3足す3は6です。そして、6に3を掛けると18です。


https://zenn.dev/tsuzukia/articles/8fc74bdb8770a5